# Visualize TimePix3 data

read from "hdf5" files created by Kuepper et al. (Hubertus Bromberger)

### Imports

In [1]:
# %matplotlib widget
%pylab qt   

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np  
import matplotlib.pyplot as plt
import h5py
import os.path
import src.pyabel_polar
import scipy

### Utility function

In [25]:
def data_from_hdf(hdf_file_complete_path, event_type = 'raw'):
    '''Read data from TimePix HDF files
    Choose raw or centroided data, default is centroided data'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        tof = h_file[str(event_type)+'/tof'][:]
        x_pos = h_file[str(event_type)+'/x'][:]
        y_pos = h_file[str(event_type)+'/y'][:]
    print('Number of FEL pulses in the file:',len(tof))
    return tof, x_pos, y_pos

def data_sliced_by_tof(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, event_type = 'raw'):
    '''Slice data with respect to time-of-flight dimension'''
    tof, x_pos, y_pos = data_from_hdf(hdf_file_complete_path, event_type)
    sliced_x_pos = x_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_y_pos = y_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_tof = tof[np.logical_and(tof > tof_start, tof < tof_end)]
    return sliced_tof, sliced_x_pos, sliced_y_pos

def reduce_raw_data(tof, x_pos, y_pos, number_of_events):
    '''Reduce data for visualization'''
    return tof[:number_of_events], x_pos[:number_of_events],y_pos[:number_of_events]

def tof_conversion(tof, time_unit):
    '''Convert time axis'''
    if time_unit == None:
        return tof, 's'
    if time_unit == 'milli':
        return tof*10**3, 'ms'
    if time_unit == 'micro':
        return tof*10**6, 'us'

def plot_tof(tof, tof_start = None, tof_end = None, hist_bins = 100, time_unit = None):     
    '''Plot time-of-flight spectrum via numpy histogram'''
    if tof_start == None:
        tof_start = np.min(tof)
    if tof_end == None:
        tof_end = np.max(tof)
        
    print(tof_start)
    print(tof_end)
    
    fig = plt.subplots(num = 1)
    plt.clf()
    tof_start, time_tof_unit = tof_conversion(tof_start, time_unit)
    tof_end, time_tof_unit = tof_conversion(tof_end, time_unit)
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    hist_y, hist_x = np.histogram(tof, bins = hist_bins, range = (tof_start, tof_end))
    hist_y = np.append(hist_y,0)
    plt.plot(hist_x, hist_y)
    plt.title('histogram: time-of-flight')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('number of events')
    plt.show()
    return hist_x, hist_y
        
def plot_1d_histograms(tof, x_pos, y_pos, time_unit = None):      
    '''1D plots - detector position vs time-of-flight for each dimension'''
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    fig = plt.figure(num = 2)
    plt.clf()
    plt.plot(tof, x_pos,'.')
    plt.title("ToF vs x_pos")
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('x_pos [px]')
    
    fig = plt.figure(num=3)
    plt.clf()
    plt.plot(tof, y_pos,'.')
    plt.title("ToF vs y_pos")
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('y_pos [px]')
    plt.show()

def plot_2d_histograms(tof, x_pos, y_pos ,bin_tof = 6000, bin_space = 256, time_unit = None, colormin = 0, colormax = 1000):
    '''2D plot (heatmap) - detector position vs time-of-flight for each dimension'''
    
    tof, time_tof_unit = tof_conversion(tof, time_unit)

    fig = plt.figure(num = 4)
    plt.clf()
    plt.hist2d(tof, x_pos, bins = (bin_tof, np.linspace(0, bin_space, bin_space+1)), cmax=colormax)
    plt.title('2d histogram: time-of-flight / x_pos')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('x_pos')
    plt.show()
    plt.colorbar()
    
    fig = plt.figure(num = 5)
    plt.clf()
    plt.hist2d(tof, y_pos,bins= (bin_tof, np.linspace(0, bin_space, bin_space+1)), cmax=colormax)
    plt.title('2d histogram: time-of-flight / y_pos')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('y_pos')
    plt.show()
    plt.colorbar()
    
def vmi_image(x_pos, y_pos, show_image = True):
    '''Display VMI image - cmax empirically found to surpress hot pixel '''
    fig = plt.figure(num = 6)
    plt.clf()
    counts, xbins, ybins, image = plt.hist2d(x_pos, y_pos, bins=np.linspace(0, 256, 257)) #, cmax= 1000)
    plt.xlabel('x_pos [px]')
    plt.ylabel('y_pos [px]')
    if show_image == False:
        plt.close()
    return counts
    
def display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, hist_bins = 100, time_unit = None, event_type = 'raw'):
    '''Display VMI Image and time-of-flight spetrum'''
    tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start, tof_end, event_type)
    plot_tof(tof, hist_bins, time_unit)
    vmi_image(x_pos,y_pos)

    
def transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius ):
    
    counts = vmi_image(x_pos, y_pos, show_image = False)
    image_cart = np.flipud(counts.transpose())
    
    image_polar, r_grid, theta_grid = src.pyabel_polar.reproject_image_into_polar(image_cart, origin=(x_center,y_center))
    radial_ave = np.sum(image_polar, axis=1)
    
    fig = plt.figure(num = 6)
    plt.clf()
    plt.imshow(image_cart)
    plt.scatter(x_center, y_center, color='r')
    plt.gcf().gca().add_artist(plt.Circle((x_center, y_center), radius, color='r', fill=False))
    plt.xlabel('x_posr [px]')
    plt.ylabel('y_posr [px]')
    plt.title('VMI image')

    fig = plt.figure(num = 7)
    plt.clf()
    plt.imshow(image_polar)
    plt.title('Image - polar coordinates')

    fig = plt.figure(num = 8)
    plt.clf()
    plt.plot(radial_ave,'r-')
    plt.title('Radial average')
    plt.xlabel('r [px]')
    plt.ylabel('counts')
    plt.show() 

def number_of_trains_from_hdf(hdf_file_complete_path):
    '''Retrun number of recorded FEL trains in HDF file'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        trains = len(h_file['tpx3Times/triggerNr'][:])
    return trains


### Define HDF path & file

In [10]:
hdf_file_path = '/gpfs/bl1/current/processed/timepix_hdf/'
#run_number = 450   # run with IR only to see funny polarization
run_number = 562

file_start = "run_"+str(run_number).zfill(4)
file_end = 'rawOnly.hdf5'
hdf_file = [i for i in os.listdir(hdf_file_path) if os.path.isfile(os.path.join(hdf_file_path,i)) and i.startswith(file_start) and not i.endswith(file_end)][0]
hdf_file_complete_path = hdf_file_path+hdf_file
assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'

### Show complete time-of-flight
100 ms until next FEL trigger

In [11]:
tof, x_pos, y_pos = data_from_hdf(hdf_file_complete_path)

plot_tof(tof, hist_bins = 1000);

Number of FEL pulses in the file: 5102012
1.2762484402628615e-08
0.10011942838747245


### Reduce data to one single FEL train

In [12]:
tof_start = 0E-6
tof_end = 12E-6

tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)


Number of FEL pulses in the file: 5102012


### Show time-of-flight spectrum

In [13]:
plot_tof(tof, hist_bins = 5000, time_unit = 'micro');

1.2762484402628615e-08
1.1998700017556985e-05


### Show X,Y position vs time-of-flight (1d)

In [8]:
plot_1d_histograms(tof, x_pos, y_pos, time_unit = 'micro')

### Show X,Y position vs time-of-flight (2d)

In [19]:
plot_2d_histograms(tof, x_pos, y_pos, time_unit = 'micro', colormin = 0 ,colormax = 100)

### Show VMI for time-of-flight interval

In [ ]:
tof_start = 0E-6
tof_end = 12E-6

tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)
plot_tof(tof, hist_bins = 1000, time_unit = 'micro');

In [ ]:
tof_start = 5.71E-6
tof_end = 6E-6

display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start, tof_end, event_type = 'raw',time_unit = 'micro')

### Transform to polar coordinates and radial averaging

In [ ]:
x_center, y_center = 128, 120
radius = 45

tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)
transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius)

In [ ]:
plt.plot(tt1[0],tt1[1]/z1, label = 'Run 411, I_krum = 20 ADU, 4127 pulses')
plt.plot(tt2[0],tt2[1]/z2, label = 'Run 423, I_krum = 220 ADU, 4592 pulses')
plt.legend()
plt.xlabel('tof [mu s]')
plt.ylabel('counts')
plt.title('0.15 laser, FEL (2xZr 300nm)')

In [ ]:
print(z1)
print(z2)